In [1]:
from langchain_docling import DoclingLoader
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

/home/aasmund_mjos/Downloads/sintef/mrst-rag-project/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
FILE_PATH = "https://www.sintef.no/projectweb/mrst/publications/papers-by-mrst-team/"
loader = DoclingLoader(file_path=FILE_PATH)
docs = loader.load()

In [29]:
print(docs[0].metadata.get('dl_meta').get('doc_items'))

[{'self_ref': '#/texts/2', 'parent': {'$ref': '#/groups/2'}, 'children': [], 'content_layer': 'body', 'label': 'list_item', 'prov': []}, {'self_ref': '#/texts/3', 'parent': {'$ref': '#/groups/2'}, 'children': [], 'content_layer': 'body', 'label': 'list_item', 'prov': []}, {'self_ref': '#/texts/4', 'parent': {'$ref': '#/groups/2'}, 'children': [], 'content_layer': 'body', 'label': 'list_item', 'prov': []}, {'self_ref': '#/texts/5', 'parent': {'$ref': '#/groups/2'}, 'children': [], 'content_layer': 'body', 'label': 'list_item', 'prov': []}, {'self_ref': '#/texts/6', 'parent': {'$ref': '#/groups/2'}, 'children': [], 'content_layer': 'body', 'label': 'list_item', 'prov': []}, {'self_ref': '#/texts/7', 'parent': {'$ref': '#/groups/2'}, 'children': [], 'content_layer': 'body', 'label': 'list_item', 'prov': []}, {'self_ref': '#/texts/8', 'parent': {'$ref': '#/groups/2'}, 'children': [], 'content_layer': 'body', 'label': 'list_item', 'prov': []}, {'self_ref': '#/texts/9', 'parent': {'$ref': '#

In [64]:
response = requests.get(FILE_PATH)
soup = BeautifulSoup(response.content, 'html.parser')

documents = []

paper_lists = soup.find_all('div', class_ = "rich-text-field")[1]
for article in paper_lists.find_all("li"):
    links = []
    text = article.get_text(strip = True)
    for link in article.find_all('a'):
        links.append(link.get('href'))
    df ={"text": text, "links": links}
    documents.append(df)

In [71]:
FILE_PATH = "https://www.cambridge.org/core/services/aop-cambridge-core/content/view/7AC2425C73F6F729DB88DB1A504FA1E7/9781316519967AR.pdf/Advanced_Modeling_with_the_MATLAB_Reservoir_Simulation_Toolbox.pdf?event-type=FTLA"
loader = DoclingLoader(file_path=FILE_PATH)
docs = loader.load()

Token indices sequence length is longer than the specified maximum sequence length for this model (930 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
from chapters_to_authors import advanced_section_to_author, introduction_section_to_author

def get_number(title):
    n_chars = len(title)
    number = ""
    n_dots = 0
    for i in range(n_chars):
        c = title[i]
        if c.isdigit() or c==".":
            number+=c
            if c==".":
                n_dots += 1
        else:
            rest_of_title = title[i:]
            return number, n_dots, rest_of_title
    return number, n_dots, title
    

def sort_book(book_df, name_df, metadata_df, file_name):
    numbers = book_df.keys()
    content = book_df.values()
    section_name = name_df.values()
    metadata = metadata_df.values()
    df = pd.DataFrame()
    numbers = [[int(i) for i in num.split(".")] for num in numbers]
    numbers = np.array(numbers)
    for i in range(3):
        df[str(i)] = numbers[:,i]

    if file_name == "Advanced Book":
        section_to_author_dict = advanced_section_to_author
    if file_name == "Introduction Book":
        section_to_author_dict = introduction_section_to_author

    df['authors'] = df['0'].apply(lambda x: section_to_author_dict.get(x))
    df['content'] = content
    df['section_name'] = section_name
    df['metadata'] = metadata
    df['file_type'] = file_name
    return df.sort_values(by = ['0','1','2'])

def generate_df(book_documents, file_name):
    sections = set()
    book_df = {}
    name_df = {}
    metadata_df = {}

    max_dots = 2

    for doc in book_documents:
        pg_content = doc.page_content
        split = pg_content.split("\n")
        title = split[0]
        content = "\n".join(split[1:])
        number, n_dots, name_title = get_number(title)
        if number != "" and int(number.split(".")[0]) <= 15:

            number += ".0"*(max_dots-n_dots)

            if number not in sections:
                sections.add(number)
                book_df[number] = content
                name_df[number] = name_title
                metadata_df[number] = doc.metadata
            else:
                book_df[number] += content

    advanced_book_df = sort_book(book_df, name_df, metadata_df, file_name)
    return advanced_book_df

In [213]:
advanced_book_df = generate_df(book_documents = docs, file_name = "Advanced Book")

In [217]:
advanced_book_df.to_pickle("advanced_book_df.pkl")

In [190]:
FILE_PATH = "https://www.cambridge.org/core/services/aop-cambridge-core/content/view/F48C3D8C88A3F67E4D97D4E16970F894/9781108492430AR.pdf/An_Introduction_to_Reservoir_Simulation_Using_MATLAB_GNU_Octave.pdf?event-type=FTLA"
loader = DoclingLoader(file_path=FILE_PATH)
docs_v2 = loader.load()

Token indices sequence length is longer than the specified maximum sequence length for this model (639 > 512). Running this sequence through the model will result in indexing errors


In [215]:
introduction_book_df = generate_df(book_documents = docs_v2, file_name = "Introduction Book")

In [218]:
introduction_book_df.to_pickle("introduction_book_df.pkl")

In [20]:
chapter_df = dict()
f = open('advanced_book_dict.txt', 'r')
for line in f:
    line_list = line.split(":")
    chapter = line_list[0]
    title = line_list[1]
    chapter_df[int(chapter)] = title

In [19]:
new_df = pd.DataFrame(columns=["0"])

author = []
title = []
first = []
second = []
third = []
content = []
metadata = []
file_type = []
embedding = []
sections = []
new_df = []

for i in range(1,15):
    chapter = i
    author.append(advanced_section_to_author.get(i))
    title.append(chapter_df.get(i))
    first.append(chapter)
    second.append(0)
    third.append(0)
    content.append("")
    metadata.append(None)
    file_type.append("Advanced Book")
    embedding.append(None)
    sections.append(str(i))

new_df = pd.DataFrame({"0": first})
new_df['1'] = second
new_df['2'] = third
new_df['authors'] = author
new_df['content'] = content
new_df['title'] = title
new_df['metadata'] = metadata
new_df['file_type'] = file_type
new_df['embedding'] = embedding
new_df['sections'] = sections
new_df

,0,1,2,authors,content,title,metadata,file_type,embedding,sections
0,1,0,0,"[Runar L. Berge, Øystein S. Klemetsdal, Knut-A...",,Unstructured PEBI Grids Conforming to Lower-Di...,None,Advanced Book,None,1
1,2,0,0,"[Mohammed Al Kobaisi, Wenjuan Zhang]",,Nonlinear Finite-Volume Methods for the Flow E...,None,Advanced Book,None,2
2,3,0,0,"[Øystein S. Klemetsdal, Knut-Andreas Lie]",,Implicit Discontinuous Galerkin Methods for Tr...,None,Advanced Book,None,3
3,4,0,0,"[Knut-Andreas Lie, Olav Møyner]",,Multiscale Pressure Solvers for Stratigraphic ...,None,Advanced Book,None,4
4,5,0,0,[Olav Møyner],,Better AD Simulators with Flexible State Funct...,None,Advanced Book,None,5
5,6,0,0,[Olav Møyner],,Faster Simulation with Optimized Automatic Dif...,None,Advanced Book,None,6
6,7,0,0,"[Xin Sun, Knut-Andreas Lie, Kai Bao]",,Using State Functions and MRST’s AD-OO Framewo...,None,Advanced Book,None,7
7,8,0,0,[Olav Møyner],,Compositional Simulation with the AD-OO Framew...,None,Advanced Book,None,8
8,9,0,0,"[Daniel Wong, Florian Doster, Sebastian Geiger]",,Embedded Discrete Fracture Models\n,None,Advanced Book,None,9
9,10,0,0,"[Olufemi Olorode, Bin Wang, Harun Ur Rashid]",,Numerical Modeling of Fractured Unconventional...,None,Advanced Book,None,10


In [49]:
fname = '../datasets/book_embeddings.pkl'
authors = set()

df = pd.read_pickle(fname)
for a_list in df['authors'].tolist():
    for a in a_list:
        authors.add(a)

def generate_possible_names(author):

    name_split = author.split()
    
    new_name_split = []
    for n in name_split:
        new_name_split.extend(n.split("-"))

    name_length = len(new_name_split)
    
    max_length = 0
    for n in new_name_split:
        max_length += len(n)

    user_name_array = np.zeros((2**name_length), dtype = f'U{max_length}')
    
    iterators = {}

    def iterators_to_number():
        num = 0
        for i in range(name_length):
            if iterators[i]:
                num += 2**i
        return num
    
    def iterators_to_name():
        name = ""
        for i in range(name_length):
            n = new_name_split[i]
            if iterators[i]:
                name += n[0].lower()
            else:
                name += n.lower()
        return name

    def assign_number(depth):
        if depth < name_length-1:
            iterators[depth] = 0
            assign_number(depth+1)
            iterators[depth] = 1
            assign_number(depth+1)
        elif depth < name_length:
            iterators[depth] = 0
            user_name_array[iterators_to_number()] = iterators_to_name()
            iterators[depth] = 1
            user_name_array[iterators_to_number()] = iterators_to_name()
    depth = 0
    assign_number(depth)
    user_name_set = set([str(i) for i in list(user_name_array)])
    return user_name_set

user_name_dict = {}

for a in authors:
    u_n = generate_possible_names(a)
    user_name_dict[a] = u_n

already = set()

for user_names in user_name_dict.values():
    for user_name in user_names:
        if user_name in already:
            print("Uæææ")
        already.add(user_name)

opposite_dict = {}
for k in user_name_dict.keys():
    for v in user_name_dict[k]:
        opposite_dict[v] = k

keys = list(opposite_dict.keys())
values = list(opposite_dict.values())

author_df = pd.DataFrame({"username": keys, "name": values})
df.to_pickle('username_dataset.pkl')